In [22]:
import numpy as np
from scipy.stats import norm # to get the cumulative standard normal distribution
import matplotlib.pyplot as plt

In [20]:
def fixed_call(a):
  return np.maximum(a-K, 0)

def scheme(n,N,dt,r,sigma,T,X0,K): 

  BM = np.zeros((n,N+1)) # simulation of the B-M 
  int_W = np.zeros((n,N)) # the integral of W_t - W_ti
  inc_W = np.zeros((n,N)) # increment of B-M
  X = np.zeros((n,N+1)) # the underlying risky asset
  X[:,0] = X0

  for i in range(N):
    Z = np.random.randn(n)
    Z_ = np.random.randn(n)
    BM[:,i+1] = BM[:,i] + np.sqrt(dt)*Z 
    inc_W[:,i] = np.sqrt(dt)*Z # increment of B-M between ti and ti+1
    X[:,i+1] = X[:,i]*np.exp(sigma*inc_W[:,i] + (r-0.5*sigma**2)*dt)
    int_W[:,i] = 0.5*dt*inc_W[:,i] + np.sqrt((dt**3)/12)*Z_ # the integral of W_t - W_ti between ti and ti+1 that is distributed as a N((W_ti+1-W_ti)*h/2 , h**3/12)

  return BM, inc_W, X, int_W

In [8]:
r, sigma, T, X0, K = 0.1, 0.2, 1, 100, 100  # the 'exact' price of an asian call as given by B. Bouchard for these values is 7.04
N = 50 # discretization
dt = T/N
n = 10**5 # mc simulation
S = scheme(n,N,dt,r,sigma,T,X0,K) 

# Standard schemes :

###Euler (i.e. Riemann Sum)

In [9]:
def euler(scheme): 
  X = scheme[2]
  A = (dt/T)*np.sum(X[:,:-1], axis=1) 
  V = np.exp(-r*T)*fixed_call(A)
  p,s = np.mean(V), np.std(V)
  return p,s

p,s = euler(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 6.91638104499745
Standard deviation: 0.026552405207867495
Confidence interval 95%: [6.864338330790029, 6.96842375920487]
Error: 0.7524558561599534 %


###Trapezoidal

In [10]:
def trapezoidal(scheme):
  X = scheme[2]
  A = (dt/T)*np.sum(X[:,:-1] + X[:,1:], axis=1)/2
  V = np.exp(-r*T)*fixed_call(A)
  p,s = np.mean(V), np.std(V)
  return p,s

p,s = trapezoidal(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 7.03943064753045
Standard deviation: 0.027002695913256502
Confidence interval 95%: [6.986505363540467, 7.092355931520433]
Error: 0.7518404064190876 %


###Trapezoidal with a Taylor expansion

In [11]:
def eq_trapezoidal(scheme):
  inc_W, X = scheme[1], scheme[2]
  taylor_expansion = X[:,:-1]*(1 + r*dt*0.5 + sigma*0.5*inc_W)
  A = (dt/T)*np.sum(taylor_expansion, axis=1)
  V = np.exp(-r*T)*fixed_call(A)
  p,s = np.mean(V), np.std(V)
  return p,s

p,s = eq_trapezoidal(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 7.039313389567919
Standard deviation: 0.02700104166090436
Confidence interval 95%: [6.986391347912547, 7.092235431223291]
Error: 0.7518068698831003 %


#Higher accuracy schemes :

### Scheme with a developpement under the B-S model

In [12]:
def dl_bs(scheme):
  X, int_W = scheme[2], scheme[3]
  taylor_expansion = X[:,:-1]*(dt + 0.5*r*dt**2 + sigma*int_W)
  A = (1/T)*np.sum(taylor_expansion, axis=1)
  V = np.exp(-r*T)*fixed_call(A)
  p,s = np.mean(V), np.std(V)
  return p,s

p,s = dl_bs(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 7.038712650803664
Standard deviation: 0.027000289101191212
Confidence interval 95%: [6.985792084165329, 7.0916332174419985]
Error: 0.7518500791802096 %


#Variance reduction :

### Expectation of the control variable

In [13]:
d_ = 0.5*(r-(sigma**2)/6)*T
d = (np.log(X0/K) + 0.5*(r+(sigma**2)/6)*T) / (sigma*np.sqrt(T/3)) 
expectation = np.exp(-r*T)*(X0*norm.cdf(d)*np.exp(d_) - K*norm.cdf(d-sigma*np.sqrt(T/3))) # (discounted) expextation of the control variable 

### Euler

In [14]:
def euler_vr(scheme): 
  W = scheme[0]
  X = scheme[2]
  Z = X0*np.exp( (r-(sigma**2)/2)*T/2 + (sigma/T)*dt*np.sum(W[:,:-1], axis=1) ) # control variable
  A = (dt/T)*np.sum(X[:,:-1], axis=1)  
  V = np.exp(-r*T)*(fixed_call(A) - fixed_call(Z))
  p,s = np.mean(V) + expectation , np.std(V)
  return p,s

p,s = euler_vr(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 6.988375517059082
Standard deviation: 0.0012001996636765982
Confidence interval 95%: [6.986023125718276, 6.990727908399888]
Error: 0.033661490214196295 %


### Trapezoidal

In [15]:
def trapezoidal_vr(scheme):
  X, W = scheme[2], scheme[0]
  Z = X0*np.exp( (r-(sigma**2)/2)*T/2 + (sigma/T)*dt*0.5*np.sum(W[:,:-1]+W[:,1:], axis=1) ) # Control variable
  A = (dt/T)*np.sum(X[:,:-1] + X[:,1:], axis=1)/2
  V = np.exp(-r*T)*(fixed_call(A) - fixed_call(Z))
  p,s = np.mean(V) + expectation , np.std(V)
  return p,s

p,s = trapezoidal_vr(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 7.040328608710973
Standard deviation: 0.0012694754823706956
Confidence interval 95%: [7.037840436765526, 7.04281678065642]
Error: 0.0353417018399959 %


### Trapezoidal with a Taylor expansion

In [16]:
def eq_trapezoidal_vr(scheme):
  inc_W, X, W = scheme[1], scheme[2], scheme[0]
  Z = X0*np.exp( (r-(sigma**2)/2)*T/2 + (sigma/T)*dt*0.5*np.sum(W[:,:-1]+W[:,1:], axis=1) ) # Control variable
  taylor_expansion = X[:,:-1]*(1 + r*dt*0.5 + sigma*0.5*inc_W)
  A = (dt/T)*np.sum(taylor_expansion, axis=1)
  V = np.exp(-r*T)*(fixed_call(A) - fixed_call(Z))
  p,s = np.mean(V) + expectation , np.std(V)
  return p,s

p,s = eq_trapezoidal_vr(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 7.040211350748441
Standard deviation: 0.0012671167936156135
Confidence interval 95%: [7.037727801832955, 7.042694899663927]
Error: 0.0352766244045014 %


### Scheme with a developpement under the B-S model

In [17]:
def dl_bs_vr(scheme):
  X, int_W, W = scheme[2], scheme[3], scheme[0]
  int_BM = int_W + W[:,:-1]*dt # integral of W on t_i t_i+1
  Z = X0*np.exp( (r-(sigma**2)/2)*T/2 + (sigma/T)*np.sum(int_BM, axis=1) ) # Control variable
  taylor_expansion = X[:,:-1]*(dt + 0.5*r*dt**2 + sigma*int_W)
  A = (1/T)*np.sum(taylor_expansion, axis=1)
  V = np.exp(-r*T)*(fixed_call(A) - fixed_call(Z))
  p,s = np.mean(V) + expectation, np.std(V)
  return p,s

p,s = dl_bs_vr(S)

print("Real value:", 7.04)
print("Estimator:", p)
print("Standard deviation:",s/np.sqrt(n))
print("Confidence interval 95%:",[p-(1.96*s)/np.sqrt(n), p+(1.96*s)/np.sqrt(n)])
print("Error:",100*1.96*s/(p*np.sqrt(n)),"%")

Real value: 7.04
Estimator: 7.040156282613127
Standard deviation: 0.0012675703627218018
Confidence interval 95%: [7.037671844702192, 7.042640720524062]
Error: 0.035289527834353296 %
